In [179]:
import pandas as pd
import numpy as np
import datetime as dt
from WindPy import w

In [180]:
w.start()

.ErrorCode=0
.Data=[Already conntected!]

In [181]:
today = (dt.date.today() - dt.timedelta(1)).strftime('%Y%m%d')

In [182]:
today

'20200714'

In [183]:
DATA_DIR = '\\\\hg-pubshare\HFM各部门共享\资产配置投资研究中心\张炀\股票池'

In [184]:
codes = pd.read_excel('%s/股票代码名单.xlsx'%(DATA_DIR), skiprows=4, index_col=0)

In [185]:
codes.columns = ['股票代码', '股票名称']

In [186]:
hkcodes = pd.read_excel('%s/港股股票代码名单.xlsx'%(DATA_DIR), skiprows=4, index_col=0)

In [187]:
hkcodes.columns = ['股票代码', '股票名称']

In [188]:
codes = codes.merge(hkcodes, on=['股票名称'], how='outer')

In [189]:
codes['股票代码'] = [codes.loc[i, '股票代码_y'] if pd.isna(codes.loc[i, '股票代码_x']) else codes.loc[i, '股票代码_x'] for i in codes.index]

In [190]:
codes = codes[['股票名称', '股票代码']]

In [191]:
stocks = pd.read_excel('%s/股票池.xlsx'%(DATA_DIR))

In [192]:
results = stocks.merge(codes, how='left', on=['股票名称'])

In [196]:
stock2code = {s: c for s, c in zip(codes['股票名称'], codes['股票代码'])}

In [67]:
data = w.wss(','.join(results['股票代码'].dropna().tolist()),
             "close,eps_ttm,pe_ttm,val_pe_deducted_ttm,pe_lyr,ps_ttm","tradeDate=%s;priceAdj=U;cycle=D"%(today))

In [70]:
info = pd.DataFrame(data.Data, 
                    index=['收盘价', 'eps(ttm)', 'pe(ttm)', '扣非pe(ttm)', 'pb(lyr)', 'ps(ttm)'],
                    columns=data.Codes)
info = info.T

In [72]:
info['股票代码'] = info.index

In [73]:
info.head()

,收盘价,eps(ttm),pe(ttm),扣非pe(ttm),pb(lyr),ps(ttm),股票代码
600383.SH,15.01,2.186130,6.866013,8.555642,6.725783,1.142076,600383.SH
601901.SH,9.03,0.097033,93.061333,101.456372,73.775504,11.118667,601901.SH
601398.SH,5.14,0.883018,5.820949,5.851385,5.867352,2.163758,601398.SH
601601.SH,31.57,3.185958,9.909108,11.762749,10.948206,0.777664,601601.SH
600036.SH,36.51,3.783608,9.649521,9.692079,9.915003,3.317313,600036.SH


In [75]:
results = results.merge(info, how='left', on=['股票代码'])

In [76]:
new_columns = stocks.columns[:2].tolist() + results.columns[-7:].tolist() + stocks.columns[2:].tolist()

In [77]:
results = results[new_columns]

In [78]:
results

,主题行业,股票名称,股票代码,收盘价,eps(ttm),pe(ttm),扣非pe(ttm),pb(lyr),ps(ttm),上次预测时间,最新短评,最新长评,内部股票池（40%）,外部股票池（20%）,基金重仓股票池（20%）,北向资金重仓股票池（20%）,总分
0,金融地产,金地集团,600383.SH,15.01,2.186130,6.866013,8.555642,6.725783,1.142076,2020-03-11,2.0,2.0,NaN,NaN,1.0,NaN,0.2
1,金融地产,方正证券,601901.SH,9.03,0.097033,93.061333,101.456372,73.775504,11.118667,2019-12-27,2.0,2.0,NaN,NaN,NaN,1.0,0.2
2,金融地产,工商银行,601398.SH,5.14,0.883018,5.820949,5.851385,5.867352,2.163758,2020-04-29,2.0,2.0,NaN,NaN,1.0,NaN,0.2
3,金融地产,中国太保,601601.SH,31.57,3.185958,9.909108,11.762749,10.948206,0.777664,2019-12-27,2.0,1.0,NaN,NaN,1.0,NaN,0.2
4,金融地产,招商银行,600036.SH,36.51,3.783608,9.649521,9.692079,9.915003,3.317313,2020-04-30,1.0,1.0,1.0,1.0,1.0,NaN,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,其他经济不敏感,上海机场,600009.SH,72.39,1.930292,37.502106,37.477388,27.730954,14.224410,2020-04-28,2.0,1.0,NaN,NaN,1.0,1.0,0.4
376,其他经济不敏感,深圳机场,000089.SZ,9.82,0.146090,67.218834,60.476750,33.919472,5.828463,2020-04-29,2.0,2.0,NaN,NaN,NaN,1.0,0.2
377,其他经济不敏感,新希望,000876.SZ,33.72,1.423732,23.684240,22.786182,28.195960,1.645283,2020-04-29,2.0,1.0,1.0,1.0,1.0,NaN,0.8
378,其他经济不敏感,中国联通,600050.SH,5.69,0.153191,37.143124,33.547355,35.433997,0.606251,2019-12-28,2.0,2.0,NaN,NaN,1.0,NaN,0.2
